In [ ]:
pip install pytelegrambotapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 5.2 MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.7.0-py3-none-any.whl size=192826 sha256=4262286bac8c917a27fd4775edfb8035ea2858fb0076055c757d8b4ca28f1444
  Stored in directory: /root/.cache/pip/wheels/ba/13/da/8abf941f7cf9f993cde6118a56a7c24e12ed791507acd8ea39
Successfully built pytelegrambotapi


In [ ]:
import telebot as tb
import requests as rq
import random as rd


# Конечная точка сайта с голосами птиц для России
birds_url = "https://www.xeno-canto.org/api/2/recordings?query=cnt:Russian"
wikiEndPoint = 'https://en.wikipedia.org/api/rest_v1/page/'
username = 'koral5819@yandex.ru'
password = 'D#-R723wLcV'

bot_token = "5621803997:AAGsNlOxN6rfxkFS_18TzcnTpmf6LWAa0mc" # токен, полученный при создании бота

class Bird:
  def __init__(self, birdName_en, soundUrl):
    self.soundsUrl = [] # перечень песен птицы
    self.addSound(soundUrl)
    self.birdName_en = birdName_en # Наименование птицы Engl
    self.birdViewUrl = '' # URL фотографии птицы

  # задает URL фотографии птицы
  def setBirdView(self, birdViewUrl):
    self.birdViewUrl = birdViewUrl

  # Возвращает URL фотографии птицы
  def getBirdView(self):
    return self.birdViewUrl

  # добавляет очередную песню в список песен птицы
  def addSound(self, sndUrl):
    self.soundsUrl.append(sndUrl)
  
  # возвращает песню птицы выбранную случайным образом из имеющихся для этой птицы песен
  def getBirdsSongUrl(self):
    return self.songUrl[rd.randint(0, len(self.songUrl) - 1)]

bot = tb.TeleBot(bot_token) # Создание экземпляра класса TeleBot

# Регистрация функции в качестве обработчика команды /start
@bot.message_handler(commands=["start"])
def start(message):
    birds = BirdsData(username, password,birds_url)
    birds.getData(0)
    bot.send_message(message.chat.id, birds.getBirdsSongUrl())


class BirdREST_API:
  def __init__(self, user, psw, endPoint, wikiEndPoint, qtyBirds = 5):
    self.params = {"page": 1} # настраиваемые параметры запроса для сайта птиц
    self.user = user # Пользователь для сайта птиц
    self.psw = psw # Пароль пользователя для сайта птиц
    self.url = endPoint # Конечная точка для сайта птиц
    self.page = 1 # Стартовая страница запроса для сайта птиц
    self.numPages = 0 # количество страниц ответа на запрос для сайта птиц
    self.qtyBirds = qtyBirds # количество птиц для викторины
    self.bird = {} # данные о птицах в виде словаря: birdName_en : bird_obj
    self.errGet = {} # словарь ошибок при выполнении запроса по страницам
    self.recBirds = {} # словарь записей страницы птиц
    self.wikiUrl = wikiEndPoint # Конечная точка для wiki
    self.smallViewUrl = '' # url меленькой фотографии птицы
    self.errGetViewBird = {} # Словарь ошибок выполнения запроса информации по птице
  
  # возвращает № страницы, выбранной случайным образом из имеющихся в запросе
  def getRandomPage(self):
    return rd.randint(1, self.numPages)

  # возвращает информацию для заданной птицы из wiki
  def getWikiInfoBird(self, birdName_en):
    self.clearBirdView() # очистка фотографии птицы
    resp = rq.get(self.wikiUrl + 'summary/' + birdName_en)
    if resp.status_code == 200:
      # Запрос выполнен успешно
      json = resp.json() # полученные json данные запроса заданной страницы
      self.smallViewUrl = json['thumbnail']['source']
      return True
    else:
      self.errGetViewBird[birdName_en] = str(resp.status_code)
      return False

  # очищает фотографию птицы
  def clearBirdView(self):
    self.smallViewUrl = ''

  # возвращает фотографию птицы
  def getBirdView(self):
    return self.smallViewUrl

  # возвращает выборку одной страницы ответа на запрос в json формате и количество страниц ответа на запрос
  def getDataBirds(self, page = 1):
    # Получаем данные с сайта
    params = {}
    params["page"] = page
    resp = rq.get(self.url, params = self.params, auth=(self.user, self.psw), allow_redirects=True)
    if resp.status_code == 200:
      # Запрос выполнен успешно
      json = resp.json() # полученные json данные запроса заданной страницы
      self.numPages = json["numPages"] # Количество страниц в запросе
      self.recBirds.clear() # предварительная очистка словаря записей птиц страницы
      self.recBirds = json['recordings']
      return True
    else:
      self.errGet[page] = str(resp.status_code)
      return False

  # заполение списка птиц для викторины случайным образом
  def getRandomLstBirds(self):
    if self.getDataBirds(self.page) == False:
      # ошибка выполнения запроса
      return False
    # ответ получен
    while len(self.bird) <  self.qtyBirds:
      # пока не занесено требуемое количество птиц
        # выбор случайной записи страницы
      record = self.recBirds[rd.randint(0, len(self.recBirds) - 1)]
      birdName_en = record["en"] # английское название птицы
      if birdName_en not in self.bird:
        # Выбранная птица еще не занесена в список викторины
        self.bird[birdName_en] = Bird(birdName_en, record["file"]) # добавление в список
        if self.getWikiInfoBird(birdName_en) == True:
          self.bird[birdName_en].setBirdView(self.getBirdView)
      self.page = self.getRandomPage() # Выбор очередной страницы
      if self.getDataBirds(self.page) == False:
        # ошибка выполнения запроса
        return False
    return True
    
  # возвращает список птиц, выбранных для викторины
  def getBirdsList(self):
    return self.bird

  # возвращает список ошибок выполнения запросов страниц
  # в формате {page: 'код ошибки: ....'}
  def getErrors(self):
    return self.errGet




# Постоянная отправка запроса на сервера Telegram (none_stop - не будет останавливаться даже
# в случае получения ошибки)
bot.polling(none_stop=True)